# From detected synch cues, produce restamped raw Equivital and fixed intervals on march 6th, Wednesday

This interval only had one recorded synch cue and no recorded audio, so the clock corrections are set according to the estimated device clock drift from other measurement days and the event timing is estimated from the identifyalbe music-coordinated actions in the signals. 

    1. Look for oboeist tuning As in respiratoin for concert timing 
    2. Look for english horn performance timed inspiration to align rating task timing
    
    
 1st Oboe is WW506, 2nd Oboe and English horn is WW508. No flutes  
 
 Moments to check for alignement and respiration: Tuning times (WW506), Ros5 (WW508), or moments within Ross 

In [18]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json 
from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate
from scipy.interpolate import interp1d

In [19]:
%load_ext autoreload
%autoreload 1
%aimport qex
%aimport al
%aimport eq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
 %reload_ext autoreload

In [79]:
# timing of events for 2024-03-07
df_timings = pd.DataFrame(columns=['Event','Event_Type','SS_sTime','SS_dTime'])
Event_data = {'Event':'','Event_Type':'','SS_sTime':0,'SS_dTime':pd.to_timedelta('nan')}
sync_start = pd.to_datetime('2024-03-06 07:12:08.000000000')



In [80]:
# 19059.7765
i=0
event_dets = Event_data.copy()
event_dets['Event']='Sync'
event_dets['Event_Type']='Sync_Start'
event_dets['SS_sTime']= 0 # actually the alignment with synch spider recording
event_dets['SS_dTime']= pd.to_datetime('2024-03-06 07:12:08.000000000').tz_localize(None)
df_timings.loc[i,:]=event_dets
i=1
event_dets = Event_data.copy()
event_dets['Event']='C1'
event_dets['Event_Type']='Concert_Start'
event_dets['SS_sTime']= (pd.to_datetime('2024-03-06 08:34:59.900000').tz_localize(None)-sync_start).total_seconds()
event_dets['SS_dTime']= pd.to_datetime('2024-03-06 08:34:59.900000').tz_localize(None) #' sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets

i=2
event_dets = Event_data.copy()
event_dets['Event']='C1'
event_dets['Event_Type']='Concert_End'
event_dets['SS_sTime']= df_timings.loc[i-1,'SS_sTime'] + 3182.218
event_dets['SS_dTime']= sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets

i=3
event_dets = Event_data.copy()
event_dets['Event']='C2'
event_dets['Event_Type']='Concert_Start' # '2024-03-06 10:14:53.800' # '2024-03-06 10:19:23.800' + 24s?
event_dets['SS_sTime']= (pd.to_datetime('2024-03-06 10:19:47.90000').tz_localize(None)-sync_start).total_seconds()
event_dets['SS_dTime']= pd.to_datetime('2024-03-06 10:19:47.90000').tz_localize(None) #' sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets

i=4
event_dets = Event_data.copy()
event_dets['Event']='C2'
event_dets['Event_Type']='Concert_End'
event_dets['SS_sTime']= df_timings.loc[i-1,'SS_sTime'] + 3214.929
event_dets['SS_dTime']= sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets

i=5
event_dets = Event_data.copy()
event_dets['Event']='WTOrate1' # '2024-03-06 11:16:51.240000'
event_dets['Event_Type']='Rating_Start'
event_dets['SS_sTime']= (pd.to_datetime('2024-03-06 11:16:51.240000').tz_localize(None)-sync_start).total_seconds()
event_dets['SS_dTime']= pd.to_datetime('2024-03-06 11:16:51.240000').tz_localize(None) #' sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets

i=6
event_dets = Event_data.copy()
event_dets['Event']='WTOrate1'
event_dets['Event_Type']='Rating_End'
event_dets['SS_sTime']= df_timings.loc[i-1,'SS_sTime']+396.366
event_dets['SS_dTime']= sync_start + pd.to_timedelta(event_dets['SS_sTime'],unit='s')
df_timings.loc[i,:]=event_dets
df_timings

,Event,Event_Type,SS_sTime,SS_dTime
0,Sync,Sync_Start,0,2024-03-06 07:12:08
1,C1,Concert_Start,4971.9,2024-03-06 08:34:59.900000
2,C1,Concert_End,8154.118,2024-03-06 09:28:02.118000
3,C2,Concert_Start,11259.9,2024-03-06 10:19:47.900000
4,C2,Concert_End,14474.829,2024-03-06 11:13:22.829000
5,WTOrate1,Rating_Start,14683.24,2024-03-06 11:16:51.240000
6,WTOrate1,Rating_End,15079.606,2024-03-06 11:23:27.606000


In [81]:
df_shifts=pd.read_csv('./outfiles/Cue_ACC_dev_align_20240306.csv', index_col=0)
df_shifts = df_shifts.loc[df_shifts['Cue1_s'].notna(),:].copy()

df_shifts['Cue1_devdTime'] = pd.to_datetime(df_shifts['Cue1_devdTime']).dt.tz_localize(None)
df_shifts

,Cue1_devdTime,Cue1_s,Cue1_r,devID
DevName,,,,
VN101,2024-03-06 07:12:07.890625000,1.109375,0.566995,3420415
VN104,2024-03-06 07:12:08.398437500,0.601562,0.453916,3420456
VN105,2024-03-06 07:12:07.121093750,1.878906,0.450958,3420755
VN109,2024-03-06 07:12:07.328125000,1.671875,0.469074,5022010
VN110,2024-03-06 07:12:08.507812500,0.492188,0.469037,5022026
VN111,2024-03-06 07:12:07.046875000,1.953125,0.447598,5022027
VN206,2024-03-06 07:12:07.792968750,1.207031,0.516876,5022062
VN201,2024-03-06 07:12:08.562500000,0.437500,0.536402,5022050
VN211,2024-03-06 07:12:07.625000000,1.375000,0.556636,5022247


In [82]:
df_timings.loc[:,'SS_sTime']

0            0
1       4971.9
2     8154.118
3      11259.9
4    14474.829
5     14683.24
6    15079.606
Name: SS_sTime, dtype: object

In [83]:
for ev in ['C1','C2','WTOrate1']:
    df_t = df_timings.query('Event == @ev').reset_index(drop = True)
    print(df_t)

  Event     Event_Type  SS_sTime                    SS_dTime
0    C1  Concert_Start    4971.9  2024-03-06 08:34:59.900000
1    C1    Concert_End  8154.118  2024-03-06 09:28:02.118000
  Event     Event_Type   SS_sTime                    SS_dTime
0    C2  Concert_Start    11259.9  2024-03-06 10:19:47.900000
1    C2    Concert_End  14474.829  2024-03-06 11:13:22.829000
      Event    Event_Type   SS_sTime                    SS_dTime
0  WTOrate1  Rating_Start   14683.24  2024-03-06 11:16:51.240000
1  WTOrate1    Rating_End  15079.606  2024-03-06 11:23:27.606000


In [84]:
def remove_tz_from_dataframe(df_in):
    df = df_in.copy()
    col_times = [ col for col in df.columns if any([isinstance(x, pd.Timestamp) for x in df[col]])]
    for col in col_times:
        df[col] = pd.to_datetime(df[col],format='ISO8601') 
        df[col] = df[col].dt.tz_localize(None) 
    return df


# Output for each event from full shifted rtime


In [85]:
concert = '20240306'
data_dir='/Volumes/TaeTae/Stavangar/bic2024/'
sep = '/'
DATAtag='EQDATA'
df_datafiles = eq.eq_recordings(data_dir+concert+'/','EQDATA','/')
df_datafiles.set_index('PartID',inplace=True)
df_datafiles

,Signal,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,EventStart,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
PartID,,,,,,,,,,,,,,,,,,,,,
VN101,DATA,[3420415],2024-03-06,20240306,VN101_20240306_FULL_EQDATA.csv,csv,149898,2024-03-06 07:01:28.014300,2024-03-06 12:19:00.845300000,0,...,[VN101],3420415.0,VN101,53.0,69,98.0,0.0,0.0,MovingSlowly,Upright
VN104,DATA,[3420456],2024-03-06,20240306,VN104_20240306_FULL_EQDATA.csv,csv,148562,2024-03-06 07:01:28.510400,2024-03-06 12:18:01.218200000,0,...,[VN104],3420456.0,VN104,53.0,83,98.0,0.0,0.0,Stationary,Upright
VN109,DATA,[5022010],2024-03-06,20240306,VN109_20240306_FULL_EQDATA.csv,csv,148708,2024-03-06 07:01:31.619800,2024-03-06 12:19:03.179000000,0,...,[VN109],5022010.0,VN109,53.0,81,98.0,0.0,0.0,Stationary,Upright
VN105,DATA,[3420755],2024-03-06,20240306,VN105_20240306_FULL_EQDATA.csv,csv,148451,2024-03-06 07:01:31.791600,2024-03-06 12:19:06.406600000,0,...,[VN105],3420755.0,VN105,53.0,0,98.0,0.0,0.0,Stationary,Upright
VN110,DATA,[5022026],2024-03-06,20240306,VN110_20240306_FULL_EQDATA.csv,csv,149022,2024-03-06 07:01:34.450000,2024-03-06 12:19:05.719700000,0,...,[VN110],5022026.0,VN110,53.0,79,98.0,0.0,0.0,Stationary,Upright
VN111,DATA,[5022027],2024-03-06,20240306,VN111_20240306_FULL_EQDATA.csv,csv,148849,2024-03-06 07:01:35.910200,2024-03-06 12:18:22.201300000,0,...,[VN111],5022027.0,VN111,53.0,80,95.0,0.0,0.0,Stationary,Upright
VN201,DATA,[5022050],2024-03-06,20240306,VN201_20240306_FULL_EQDATA.csv,csv,148894,2024-03-06 07:02:52.400700,2024-03-06 12:20:08.660000000,0,...,[VN201],5022050.0,VN201,53.0,75,97.0,0.0,0.0,Stationary,Upright
VN206,DATA,[5022062],2024-03-06,20240306,VN206_20240306_FULL_EQDATA.csv,csv,147774,2024-03-06 07:02:53.162200,2024-03-06 12:17:54.689900000,0,...,[VN206],5022062.0,VN206,53.0,66,99.0,0.0,0.0,Stationary,Upright
VN211,DATA,[5022247],2024-03-06,20240306,VN211_20240306_FULL_EQDATA.csv,csv,148189,2024-03-06 07:02:55.337300,2024-03-06 12:18:41.630300000,0,...,[VN211],5022247.0,VN211,53.0,77,97.0,0.0,0.0,Stationary,Upright


In [66]:
pid = 'VA304'
eqfiles = df_datafiles.loc[pid,['DATAloc','BACCloc','RESPloc','CIBIloc','2ECGloc']]
eqfiles

DATAloc    /Volumes/TaeTae/Stavangar/bic2024/20240306/VA3...
BACCloc    /Volumes/TaeTae/Stavangar/bic2024/20240306/VA3...
RESPloc    /Volumes/TaeTae/Stavangar/bic2024/20240306/VA3...
CIBIloc    /Volumes/TaeTae/Stavangar/bic2024/20240306/VA3...
2ECGloc    /Volumes/TaeTae/Stavangar/bic2024/20240306/VA3...
Name: VA304, dtype: object

In [69]:
f

'DATAloc'

In [86]:
CUTLOGS = pd.concat([df_datafiles,df_shifts],axis=1)#,CUTLOGS
CUTS= pd.Series()
CUTS[pid] = []
# newdata_dir='/Volumes/SeagateHub/BiC2024_Synced/Equivital/'
newdata_dir='/Volumes/TaeTae/Stavangar/bic2024/'
sig_map = {'DATAloc':'EQDATA','BACCloc':'EQBACC','RESPloc':'EQRESP','CIBIloc':'EQCIBI','2ECGloc':'EQ2ECG'}
cut_ind = 0
toc = time.time()
rec_day = '20240306'
buff = pd.to_timedelta(15,unit='s') # 2 minute buffer on cutes for concert and stuff pd.to_timedelta(RecTimeSig['rec_sTime'] - ap['Cue0_rst'],unit='s')
segmentid = 'FULL'

for pid in CUTLOGS.index:
    tic = time.time()
    outfiles = []
    tic = time.time()
#     drift_C = (df_timings.loc[1,'SS_sTime'] - df_timings.loc[0,'SS_sTime'])/(df_shifts.loc[pid,'Cue2_devdTime']-df_shifts.loc[pid,'Cue1_devdTime']).total_seconds()
    eqfiles = df_datafiles.loc[pid,['DATAloc','BACCloc','RESPloc','CIBIloc','2ECGloc']]
    for f in eqfiles.index:
        SignalID = sig_map[f]
        RecTimeSig = pd.read_csv(eqfiles[f],skipinitialspace=True)
        RecTimeSig['rec_dTime']=pd.to_datetime(RecTimeSig['rec_dTime'])
        sig_col = RecTimeSig.columns
        new_col_order=['c_sTime']+list(sig_col) + ['rec_dTime','dev_dTime']
        
        # run through events in that section
        for ev in ['C2','WTOrate1']:#['C1','C2','WTOrate1']:
            df_t = df_timings.query('Event == @ev').reset_index(drop = True)
            subsig  = al.dt_cut(RecTimeSig,'rec_dTime',df_t.loc[0,'SS_dTime']-buff,df_t.loc[1,'SS_dTime']+buff)
            subsig['c_sTime'] = np.round((subsig['rec_dTime']-df_t.loc[0,'SS_dTime']).dt.total_seconds(),4)
            #C1_FULL_BR601_EQBACC 
            seg_filename = '_'.join([ev,segmentid,pid,SignalID+'.csv'])
            outfiles.append(seg_filename)
            subsig[new_col_order].to_csv(newdata_dir + ev+'/' + seg_filename, index = False)
           
        print(' '.join([pid,SignalID ,str(np.round(time.time() - tic,3)),str(np.round(time.time() - toc,3))]))


CUTS= pd.Series()
CUTS[pid] = []
CUTLOGS['Output_files']=CUTS
CUTLOGS.to_csv('./outfiles/'+rec_day+'EQIVITALS_CUTLOGS.csv')
# CUTLOGS['Scaling_C'].hist()
# plt.show()
CUTLOGS

VN101 EQDATA 1.267 1.268
VN101 EQBACC 22.765 22.766
VN101 EQRESP 25.131 25.132
VN101 EQCIBI 25.323 25.324
VN101 EQ2ECG 50.285 50.286
VN104 EQDATA 0.294 50.58
VN104 EQBACC 22.016 72.302
VN104 EQRESP 24.042 74.328
VN104 EQCIBI 24.207 74.493
VN104 EQ2ECG 44.212 94.498
VN109 EQDATA 0.33 94.829
VN109 EQBACC 21.841 116.339
VN109 EQRESP 24.023 118.521
VN109 EQCIBI 24.202 118.701
VN109 EQ2ECG 44.129 138.627
VN105 EQDATA 0.265 138.893
VN105 EQBACC 18.974 157.602
VN105 EQRESP 21.204 159.832
VN105 EQCIBI 21.353 159.981
VN105 EQ2ECG 40.81 179.438
VN110 EQDATA 0.26 179.698
VN110 EQBACC 18.887 198.325
VN110 EQRESP 21.003 200.441
VN110 EQCIBI 21.189 200.627
VN110 EQ2ECG 40.632 220.07
VN111 EQDATA 0.318 220.388
VN111 EQBACC 19.43 239.501
VN111 EQRESP 21.635 241.706
VN111 EQCIBI 21.838 241.908
VN111 EQ2ECG 40.778 260.848
VN201 EQDATA 0.271 261.12
VN201 EQBACC 17.97 278.818
VN201 EQRESP 20.045 280.894
VN201 EQCIBI 20.27 281.119
VN201 EQ2ECG 39.803 300.651
VN206 EQDATA 0.303 300.954
VN206 EQBACC 18.71 31

,Signal,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,EventStart,...,HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION,Cue1_devdTime,Cue1_s,Cue1_r,devID,Output_files
VN101,DATA,[3420415],2024-03-06,20240306,VN101_20240306_FULL_EQDATA.csv,csv,149898,2024-03-06 07:01:28.014300,2024-03-06 12:19:00.845300000,0,...,98.0,0.0,0.0,MovingSlowly,Upright,2024-03-06 07:12:07.890625000,1.109375,0.566995,3420415,NaN
VN104,DATA,[3420456],2024-03-06,20240306,VN104_20240306_FULL_EQDATA.csv,csv,148562,2024-03-06 07:01:28.510400,2024-03-06 12:18:01.218200000,0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.398437500,0.601562,0.453916,3420456,NaN
VN109,DATA,[5022010],2024-03-06,20240306,VN109_20240306_FULL_EQDATA.csv,csv,148708,2024-03-06 07:01:31.619800,2024-03-06 12:19:03.179000000,0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.328125000,1.671875,0.469074,5022010,NaN
VN105,DATA,[3420755],2024-03-06,20240306,VN105_20240306_FULL_EQDATA.csv,csv,148451,2024-03-06 07:01:31.791600,2024-03-06 12:19:06.406600000,0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.121093750,1.878906,0.450958,3420755,NaN
VN110,DATA,[5022026],2024-03-06,20240306,VN110_20240306_FULL_EQDATA.csv,csv,149022,2024-03-06 07:01:34.450000,2024-03-06 12:19:05.719700000,0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.507812500,0.492188,0.469037,5022026,NaN
VN111,DATA,[5022027],2024-03-06,20240306,VN111_20240306_FULL_EQDATA.csv,csv,148849,2024-03-06 07:01:35.910200,2024-03-06 12:18:22.201300000,0,...,95.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.046875000,1.953125,0.447598,5022027,NaN
VN201,DATA,[5022050],2024-03-06,20240306,VN201_20240306_FULL_EQDATA.csv,csv,148894,2024-03-06 07:02:52.400700,2024-03-06 12:20:08.660000000,0,...,97.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:08.562500000,0.437500,0.536402,5022050,NaN
VN206,DATA,[5022062],2024-03-06,20240306,VN206_20240306_FULL_EQDATA.csv,csv,147774,2024-03-06 07:02:53.162200,2024-03-06 12:17:54.689900000,0,...,99.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.792968750,1.207031,0.516876,5022062,NaN
VN211,DATA,[5022247],2024-03-06,20240306,VN211_20240306_FULL_EQDATA.csv,csv,148189,2024-03-06 07:02:55.337300,2024-03-06 12:18:41.630300000,0,...,97.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.625000000,1.375000,0.556636,5022247,NaN
VN212,DATA,[5022256],2024-03-06,20240306,VN212_20240306_FULL_EQDATA.csv,csv,147660,2024-03-06 07:02:55.540600,2024-03-06 12:17:56.957100000,0,...,98.0,0.0,0.0,Stationary,Upright,2024-03-06 07:12:07.417968750,1.582031,0.569802,5022256,NaN


In [ ]:
# 4099.328 for all segments